# Import Libraries & Load Dataset

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot

In [2]:
dataset = pd.read_csv("Google-Playstore.csv")
dataset

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,NaN,vnacrewit@gmail.com,"Aug 9, 2019","Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,"Sep 10, 2018","Oct 13, 2018",Everyone,NaN,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,"Feb 21, 2020","Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2312939,大俠客—熱血歸來,com.rxsj.ssjj,Role Playing,4.3,16775.0,"100,000+",100000.0,337109,True,0.0,...,http://www.4399sy.com.hk/,ssjjcomhk@gmail.com,NaN,"Jun 01, 2021",Teen,http://a.4399sy.com.hk/user/aggreement,False,False,False,2021-06-16 12:59:18
2312940,ORU Online,com.threedream.oruonline,Education,0.0,0.0,100+,100.0,430,True,0.0,...,http://www.oru.edu/,3DreamDeveloper@gmail.com,"Jan 17, 2018","Feb 02, 2018",Everyone,http://www.oru.edu/about-oru/privacy-policy.php,False,False,False,2021-06-16 12:59:19
2312941,Data Structure,datastructure.appoworld.datastucture,Education,0.0,0.0,100+,100.0,202,True,0.0,...,NaN,appoworld.official@gmail.com,"Aug 19, 2018","Aug 19, 2018",Everyone,https://appoworld.000webhostapp.com/datastruct...,False,False,False,2021-06-16 12:59:19
2312942,Devi Suktam,ishan.devi.suktam,Music & Audio,3.5,8.0,"1,000+",1000.0,2635,True,0.0,...,https://a70f78905.app-ads-txt.com,ruchisono@gmail.com,"Aug 1, 2016","May 05, 2021",Everyone,https://docs.google.com/document/d/1x-9reZuLRX...,True,False,False,2021-06-16 12:59:19


In [3]:
dataset.columns

Index(['App Name', 'App Id', 'Category', 'Rating', 'Rating Count', 'Installs',
       'Minimum Installs', 'Maximum Installs', 'Free', 'Price', 'Currency',
       'Size', 'Minimum Android', 'Developer Id', 'Developer Website',
       'Developer Email', 'Released', 'Last Updated', 'Content Rating',
       'Privacy Policy', 'Ad Supported', 'In App Purchases', 'Editors Choice',
       'Scraped Time'],
      dtype='object')

# Data Cleaning

## Cleaning Redundent Features

- `Feature <<Scraped Time>> is not important`

In [4]:
dataset = dataset.drop('Scraped Time', axis=1)

- `The dataset has 3 features for install, one is for estimated install, one is for minimum install, and the last one is for maximum install. I will delete the install, min install, and max install features and create a install_count feature that is average between min install and max install`

In [5]:
dataset['install_count'] = ( dataset['Minimum Installs'] + dataset['Maximum Installs'] )  / 2
dataset = dataset.drop(['Installs', 'Minimum Installs', 'Maximum Installs'], axis=1)

- `There is also a redundant feature called <<Free>>. This is redundant becuase the <<Price>> feature can show this feature by assigning the value to 0.0, so I will delete <<Free>> as well.`

In [6]:
dataset = dataset.drop('Free', axis=1)

- `There are a lot of currency type, so I will convert all of them to USD`

In [7]:
dataset['Currency'].unique()

array(['USD', 'XXX', 'CAD', 'EUR', 'INR', nan, 'VND', 'GBP', 'BRL', 'KRW',
       'TRY', 'RUB', 'SGD', 'AUD', 'PKR', 'ZAR'], dtype=object)

In [8]:
conversion_rates = {
    'USD': 1.0, 'CAD': 0.79, 'EUR': 1.13, 'INR': 0.013, 'VND': 0.000044, 'GBP': 1.36,
    'BRL': 0.18, 'KRW': 0.00085, 'TRY': 0.071, 'RUB': 0.013, 'SGD': 0.74, 'AUD': 0.72,
    'PKR': 0.0057, 'ZAR': 0.066, 'XXX': None, float('nan'): None
}

# Function to convert to USD
def convert_to_usd(row):
    currency = row['Currency']
    price = row['Price']
    if currency in conversion_rates and conversion_rates[currency] is not None:
        return price * conversion_rates[currency]
    else:
        return None  # Handle undefined or NaN values as needed

# Apply the conversion
dataset['price_usd'] = dataset.apply(convert_to_usd, axis=1)

dataset = dataset.drop(['Price', 'Currency'], axis=1)

## Handle Missing Values

### Price (Rplacing with median)

In [9]:
dataset['price_usd'].unique()

array([ 0.      ,       nan,  1.99    , ...,  3.041816, 26.746362,
       18.903596])

In [10]:
print(f'There are {dataset[dataset['price_usd'].isna()].shape[0]} Null values in this feature')
median_price = dataset['price_usd'].median()
dataset['price_usd'].fillna(median_price, inplace=True)

There are 1371 Null values in this feature


C:\Users\Alireza HR\AppData\Local\Temp\ipykernel_10120\116779552.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['price_usd'].fillna(median_price, inplace=True)


### App Name & Rating & Rating Count (Delete)

In [11]:
null_rating = dataset[dataset['Rating'].isna() | dataset['Rating Count'].isna()]

In [12]:
print(f"{round(null_rating.shape[0]/dataset.shape[0] *100, 2)} % of the whole dataset have Nan in these two features")

0.99 % of the whole dataset have Nan in these two features


So we will delete them

In [13]:
dataset = dataset.dropna(subset=['Rating Count', 'Rating'])

In [14]:
dataset[dataset['App Name'].isna()]

,App Name,App Id,Category,Rating,Rating Count,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,install_count,price_usd
45686,NaN,se.promedia.NA,News & Magazines,2.4,470.0,27M,4.1 and up,Bonnier News Local AB,https://www.na.se/app,kundservice.bnlo@bonniernews.se,"Aug 8, 2011","Dec 21, 2020",Everyone,https://www.na.se/info/mittmedias-personuppgif...,True,False,False,67048.5,0.0
1226953,NaN,com.ibrahimtornado.none,Entertainment,4.6,11.0,1.9M,4.1 and up,Ibrahim Tornado,NaN,ibra.ibrahim9995@gmail.com,"Mar 22, 2019","Mar 23, 2019",Everyone,https://sites.google.com/view/noneapp/privacy-...,True,False,False,225.5,0.0
1712582,NaN,com.noone,Entertainment,4.1,40.0,216k,3.0 and up,TLUTech,NaN,friedchikanye@gmail.com,"Oct 6, 2014","Oct 06, 2014",Everyone,NaN,False,False,False,1703.0,0.0
1924265,NaN,com.DN.None,Arcade,4.6,5.0,22M,4.1 and up,DNGames,NaN,nguyenthaiphihungnew@outlook.com,"Dec 30, 2019","Jan 06, 2020",Everyone,NaN,True,False,False,18.5,0.0
2117918,NaN,com.Tomkii.NULl,Arcade,4.3,6.0,10M,4.4 and up,Tomkii,NaN,tomek.rybinski@interia.pl,"Mar 17, 2021","Mar 17, 2021",Everyone,NaN,True,False,False,74.0,0.0


In [15]:
dataset = dataset.dropna(subset=['App Name'])

### App Id	& Category & Size & ... (Don't have null values)

In [16]:
dataset[dataset['App Id'].isna()]

,App Name,App Id,Category,Rating,Rating Count,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,install_count,price_usd


In [17]:
dataset[dataset['Category'].isna()]

,App Name,App Id,Category,Rating,Rating Count,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,install_count,price_usd


In [18]:
dataset[dataset['Size'].isna()]

,App Name,App Id,Category,Rating,Rating Count,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,install_count,price_usd


In [19]:
dataset[dataset['Content Rating'].isna()]

,App Name,App Id,Category,Rating,Rating Count,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,install_count,price_usd


In [20]:
dataset[dataset['Ad Supported'].isna()]

,App Name,App Id,Category,Rating,Rating Count,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,install_count,price_usd


In [21]:
dataset[dataset['In App Purchases'].isna()]

,App Name,App Id,Category,Rating,Rating Count,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,install_count,price_usd


In [22]:
dataset[dataset['Editors Choice'].isna()]

,App Name,App Id,Category,Rating,Rating Count,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,install_count,price_usd


In [23]:
dataset[dataset['install_count'].isna()]

,App Name,App Id,Category,Rating,Rating Count,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,install_count,price_usd


In [24]:
dataset[dataset['price_usd'].isna()]

,App Name,App Id,Category,Rating,Rating Count,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,install_count,price_usd


### Minimum Android (Replace with Mode)

In [25]:
print(f'There are {dataset[dataset['Minimum Android'].isna()].shape[0]} Null values in this feature')
mode = dataset['Minimum Android'].mode()[0]
dataset['Minimum Android'].fillna(mode, inplace=True)

There are 6526 Null values in this feature


C:\Users\Alireza HR\AppData\Local\Temp\ipykernel_10120\2339242675.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['Minimum Android'].fillna(mode, inplace=True)


### Developer Id & Developer Website & ... (Replace with specific value)

In [26]:
dataset['Developer Id'].fillna('Not Available', inplace=True)
dataset['Developer Website'].fillna('Not Available', inplace=True)
dataset['Released'].fillna('Not Available', inplace=True)
dataset['Last Updated'].fillna('Not Available', inplace=True)
dataset['Developer Email'].fillna('Not Available', inplace=True)
dataset['Privacy Policy'].fillna("Doesn't have Privacy Policy", inplace=True)

C:\Users\Alireza HR\AppData\Local\Temp\ipykernel_10120\3584355439.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['Developer Id'].fillna('Not Available', inplace=True)
C:\Users\Alireza HR\AppData\Local\Temp\ipykernel_10120\3584355439.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

Check for existing null values

In [27]:
nan_rows_any = dataset[dataset.isna().any(axis=1)]
nan_rows_any

,App Name,App Id,Category,Rating,Rating Count,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,install_count,price_usd


# Remove Duplicates

In [28]:
print(f"Before removing duplicates: {dataset.shape[0]}")
dataset.drop_duplicates(inplace=True)
print(f"After removing duplicates: {dataset.shape[0]}")

Before removing duplicates: 2290056
After removing duplicates: 2290056


# Standardize Formats

In [29]:
dataset['Rating Count'] = dataset['Rating Count'].astype(int)

In [30]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2290056 entries, 0 to 2312943
Data columns (total 19 columns):
 #   Column             Dtype  
---  ------             -----  
 0   App Name           object 
 1   App Id             object 
 2   Category           object 
 3   Rating             float64
 4   Rating Count       int64  
 5   Size               object 
 6   Minimum Android    object 
 7   Developer Id       object 
 8   Developer Website  object 
 9   Developer Email    object 
 10  Released           object 
 11  Last Updated       object 
 12  Content Rating     object 
 13  Privacy Policy     object 
 14  Ad Supported       bool   
 15  In App Purchases   bool   
 16  Editors Choice     bool   
 17  install_count      float64
 18  price_usd          float64
dtypes: bool(3), float64(3), int64(1), object(12)
memory usage: 303.6+ MB


In [32]:
dataset.to_csv('Cleaned_dataset.csv', index=False)